In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sfa, bqa, tools, qc, config

from importlib import reload

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
query_n = 70

test = "ds"
scale = 1
cid = "01"
stream_n = 1
desc = f"qc_query_{query_n}"
seq_id = "NA"
verbose = False
verbose_query = False
qual = None
save = True
verbose_iter = False

In [3]:
shared_timestamp = pd.Timestamp.now()  # "UTC"
shared_timestamp = str(shared_timestamp).replace(" ", "_")

results_dir, _ = tools.make_name(db="bqsf", test=test, cid=cid, kind="results",
                                 datasource="ds_100GB_05", desc=desc, ext="", 
                                 timestamp=shared_timestamp)
print("Result Folder Name:")
print(results_dir)

Result Folder Name:
/home/colin/code/bq_snowflake_benchmark/results/bqsf_ds_01_results-ds_100GB_05-qc_query_70-2020-05-28_09:53:29.294666


In [4]:
#seq = tools.tpc_stream(test=test, n=stream_n)
seq = [query_n]
skip = []
for sk in skip:
    seq.remove(sk)

In [5]:
reload(sfa)
sf = sfa.SFTPC(test=test,
               scale=scale,
               cid=cid,
               warehouse="TEST9000",
               desc=desc,
               verbose=verbose,
               verbose_query=verbose_query)

print('Using database:', sf.database)

sf.timestamp = shared_timestamp
sf.results_dir = results_dir

sf.connect()
sf.query_seq(seq=seq, seq_id=seq_id, qual=qual, save=save, verbose_iter=verbose_iter)
sf.close()

sf_results_csv_fp = sf.results_csv_fp

Using database: ds_1GB_01

Query Stream Done!
Total Time Elapsed: 0 days 00:00:02.473990



In [6]:
bq = bqa.BQTPC(test=test,
               scale=scale,
               cid=cid,
               desc=desc,
               verbose_query=verbose_query, verbose=verbose)

bq.timestamp = shared_timestamp
bq.results_dir = results_dir

bq.query_seq(seq, seq_id=seq_id, qual=qual, save=save, verbose_iter=verbose_iter)

bq_results_csv_fp = bq.results_csv_fp


Query Stream Done!
Total Time Elapsed: 0 days 00:00:02.903184



In [7]:
df, result = qc.compare_results(results_dir)

['/home/colin/code/bq_snowflake_benchmark/results/bqsf_ds_01_results-ds_100GB_05-qc_query_70-2020-05-28_09:53:29.294666/query_result_sf_70.csv']


In [8]:
df

,fp_bq,q_bq,fp_sf,q_sf
1,/home/colin/code/bq_snowflake_benchmark/result...,70,/home/colin/code/bq_snowflake_benchmark/result...,70


### RESULTS SHOULD BE True if matched!

In [9]:
seq

[70]

In [10]:
result

1    False
dtype: bool

In [13]:
type(result)

pandas.core.series.Series

In [17]:
result[1]

False

In [11]:
if len(df) == 1:
    n = 1

In [12]:
d1, d2 = qc.print_head(df=df, n=n, n0=24, n1=28)

BQ:
     total_sum s_state           s_county  rank_within_parent
0 -436455561.1      TN  Williamson County                   1
----------------------------------------
SF:
     total_sum  s_state           s_county  lochierarchy  rank_within_parent
0 -436455561.1 -9999.99           -9999.99             2                   1
1 -436455561.1       TN           -9999.99             1                   1
2 -436455561.1       TN  Williamson County             0                   1
BQ:
Empty DataFrame
Columns: [total_sum, s_state, s_county, rank_within_parent]
Index: []
----------------------------------------
SF:
Empty DataFrame
Columns: [total_sum, s_state, s_county, lochierarchy, rank_within_parent]
Index: []
BQ:
     total_sum s_state           s_county  rank_within_parent
0 -436455561.1      TN  Williamson County                   1
----------------------------------------
SF:
     total_sum  s_state           s_county  lochierarchy  rank_within_parent
0 -436455561.1 -9999.99           